# 📊 Análise de Load Test - Azure Service Bus PoC

Este notebook analisa os resultados dos testes de carga comparando:
- **Bad Producer**: Cria nova conexão a cada request (e nunca fecha!)
- **Good Producer**: Reutiliza uma única conexão

## Métricas Analisadas:
- Latência de resposta
- Throughput (requests/seg)
- Taxa de erros
- Conexões vazadas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import glob
from pathlib import Path
import numpy as np

# Configurações de visualização
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 12
sns.set_palette('husl')

# Cores para os gráficos
BAD_COLOR = '#e74c3c'  # Vermelho
GOOD_COLOR = '#27ae60'  # Verde

print('✅ Bibliotecas carregadas!')

## 1. Carregar Resultados dos Testes k6

O k6 gera dois tipos de arquivos:
- `*_raw.json` - NDJSON (cada linha é um evento) - NÃO USAR
- `*_<timestamp>.json` - JSON de resumo com métricas agregadas - USAR ESTE

In [ ]:
# Função para carregar o arquivo de resumo k6 (exclui arquivos *_raw.json)
def load_latest_k6_summary(producer_type):
    """Carrega o arquivo de resumo k6 mais recente para um tipo de producer.
    
    Args:
        producer_type: 'bad_producer' ou 'good_producer'
    """
    # Buscar arquivos com timestamp (excluir _raw.json)
    all_files = glob.glob(f'results/{producer_type}_*.json')
    summary_files = [f for f in all_files if '_raw.json' not in f]
    
    if not summary_files:
        print(f'⚠️ Nenhum arquivo de resumo encontrado para: {producer_type}')
        return None
    
    latest_file = sorted(summary_files)[-1]
    print(f'📂 Carregando: {latest_file}')
    
    with open(latest_file, 'r') as f:
        return json.load(f)

# Carregar resultados
bad_producer_results = load_latest_k6_summary('bad_producer')
good_producer_results = load_latest_k6_summary('good_producer')

if bad_producer_results:
    print(f"\n📊 Bad Producer: {bad_producer_results.get('metrics', {}).get('http_reqs', {}).get('values', {}).get('count', 0)} requests")
if good_producer_results:
    print(f"📊 Good Producer: {good_producer_results.get('metrics', {}).get('http_reqs', {}).get('values', {}).get('count', 0)} requests")

In [ ]:
# Função para extrair métricas do resultado k6
def extract_k6_metrics(result):
    if not result:
        return None
    
    metrics = result.get('metrics', {})
    
    # Extrair valores com tratamento para chaves que podem não existir
    http_reqs = metrics.get('http_reqs', {}).get('values', {})
    http_duration = metrics.get('http_req_duration', {}).get('values', {})
    http_failed = metrics.get('http_req_failed', {}).get('values', {})
    leaked = metrics.get('leaked_connections', {}).get('values', {})
    success = metrics.get('success_rate', {}).get('values', {})
    
    return {
        'total_requests': http_reqs.get('count', 0),
        'rps': http_reqs.get('rate', 0),
        'avg_latency_ms': http_duration.get('avg', 0),
        'p50_latency_ms': http_duration.get('med', 0),
        'p90_latency_ms': http_duration.get('p(90)', 0),
        'p95_latency_ms': http_duration.get('p(95)', 0),
        'min_latency_ms': http_duration.get('min', 0),
        'max_latency_ms': http_duration.get('max', 0),
        'failure_rate': http_failed.get('rate', 0),
        'leaked_connections': leaked.get('count', 0),
        'success_rate': success.get('rate', 0),
    }

bad_metrics = extract_k6_metrics(bad_producer_results)
good_metrics = extract_k6_metrics(good_producer_results)

print("\n" + "="*60)
print("           📊 MÉTRICAS EXTRAÍDAS")
print("="*60)

if bad_metrics:
    print(f"\n☠️ BAD PRODUCER:")
    for key, value in bad_metrics.items():
        print(f"   {key}: {value}")

if good_metrics:
    print(f"\n✅ GOOD PRODUCER:")
    for key, value in good_metrics.items():
        print(f"   {key}: {value}")

## 2. Tabela Comparativa

In [ ]:
if bad_metrics and good_metrics:
    comparison_df = pd.DataFrame({
        'Métrica': [
            'Total de Requests',
            'Requests/segundo',
            'Latência Média (ms)',
            'Latência p50 (ms)',
            'Latência p90 (ms)',
            'Latência p95 (ms)',
            'Latência Máxima (ms)',
            'Taxa de Falhas (%)',
            'Taxa de Sucesso (%)',
            'Conexões Vazadas'
        ],
        'Bad Producer ☠️': [
            bad_metrics['total_requests'],
            f"{bad_metrics['rps']:.3f}",
            f"{bad_metrics['avg_latency_ms']:.2f}",
            f"{bad_metrics['p50_latency_ms']:.2f}",
            f"{bad_metrics['p90_latency_ms']:.2f}",
            f"{bad_metrics['p95_latency_ms']:.2f}",
            f"{bad_metrics['max_latency_ms']:.2f}",
            f"{bad_metrics['failure_rate']*100:.1f}",
            f"{bad_metrics['success_rate']*100:.1f}",
            bad_metrics['leaked_connections']
        ],
        'Good Producer ✅': [
            good_metrics['total_requests'],
            f"{good_metrics['rps']:.3f}",
            f"{good_metrics['avg_latency_ms']:.2f}",
            f"{good_metrics['p50_latency_ms']:.2f}",
            f"{good_metrics['p90_latency_ms']:.2f}",
            f"{good_metrics['p95_latency_ms']:.2f}",
            f"{good_metrics['max_latency_ms']:.2f}",
            f"{good_metrics['failure_rate']*100:.1f}",
            f"{good_metrics['success_rate']*100:.1f}",
            good_metrics['leaked_connections']
        ]
    })
    
    print('\n📊 COMPARAÇÃO DE RESULTADOS K6\n')
    display(comparison_df)
else:
    print('⚠️ Execute os testes k6 primeiro para gerar dados!')

## 3. Gráfico de Latência

In [ ]:
if bad_metrics and good_metrics:
    fig, ax = plt.subplots(figsize=(12, 7))
    
    x = np.arange(4)
    width = 0.35
    
    bad_values = [
        bad_metrics['avg_latency_ms'],
        bad_metrics['p50_latency_ms'],
        bad_metrics['p90_latency_ms'],
        bad_metrics['max_latency_ms']
    ]
    
    good_values = [
        good_metrics['avg_latency_ms'],
        good_metrics['p50_latency_ms'],
        good_metrics['p90_latency_ms'],
        good_metrics['max_latency_ms']
    ]
    
    bars1 = ax.bar(x - width/2, bad_values, width, label='Bad Producer ☠️', color=BAD_COLOR)
    bars2 = ax.bar(x + width/2, good_values, width, label='Good Producer ✅', color=GOOD_COLOR)
    
    ax.set_xlabel('Métrica de Latência', fontsize=12)
    ax.set_ylabel('Tempo (ms)', fontsize=12)
    ax.set_title('📊 Comparação de Latência: Bad vs Good Producer', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(['Média', 'p50', 'p90', 'Máximo'])
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Adicionar valores nas barras
    for bar in bars1:
        height = bar.get_height()
        ax.annotate(f'{height:.0f}ms',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)
    
    for bar in bars2:
        height = bar.get_height()
        ax.annotate(f'{height:.0f}ms',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.savefig('results/latency_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print('💾 Gráfico salvo em: results/latency_comparison.png')
else:
    print('⚠️ Dados não disponíveis')

## 4. Gráfico de Taxa de Sucesso vs Falha

In [ ]:
if bad_metrics and good_metrics:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Bad Producer
    bad_success = bad_metrics['success_rate'] * 100
    bad_fail = bad_metrics['failure_rate'] * 100
    axes[0].pie([bad_success, bad_fail], labels=['Sucesso', 'Falha'], 
                colors=[GOOD_COLOR, BAD_COLOR], autopct='%1.1f%%',
                explode=(0, 0.1), shadow=True)
    axes[0].set_title(f'☠️ BAD PRODUCER\n{bad_metrics["total_requests"]} requests', fontsize=14, fontweight='bold')
    
    # Good Producer
    good_success = good_metrics['success_rate'] * 100
    good_fail = good_metrics['failure_rate'] * 100
    axes[1].pie([good_success, good_fail], labels=['Sucesso', 'Falha'], 
                colors=[GOOD_COLOR, BAD_COLOR], autopct='%1.1f%%',
                explode=(0, 0.1), shadow=True)
    axes[1].set_title(f'✅ GOOD PRODUCER\n{good_metrics["total_requests"]} requests', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('results/success_rate_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print('💾 Gráfico salvo em: results/success_rate_comparison.png')
else:
    print('⚠️ Dados não disponíveis')

## 5. Conexões Vazadas

In [ ]:
if bad_metrics and bad_metrics['leaked_connections'] > 0:
    leaked = bad_metrics['leaked_connections']
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    categories = ['Conexões\nVazadas', 'Memória\nEstimada (MB)', 'Threads\nExtras']
    values = [leaked, leaked * 2, leaked * 3]
    colors = [BAD_COLOR, '#e67e22', '#9b59b6']
    
    bars = ax.bar(categories, values, color=colors, edgecolor='black', linewidth=2)
    
    ax.set_ylabel('Quantidade', fontsize=12)
    ax.set_title('☠️ IMPACTO DO RESOURCE LEAK - Bad Producer', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Adicionar valores
    for bar, val in zip(bars, values):
        ax.annotate(f'{val:.0f}',
                    xy=(bar.get_x() + bar.get_width() / 2, bar.get_height()),
                    xytext=(0, 5),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('results/resource_leak_impact.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f'\n☠️ IMPACTO DO VAZAMENTO:')
    print(f'   • Conexões nunca fechadas: {leaked}')
    print(f'   • Memória vazada estimada: ~{leaked * 2} MB')
    print(f'   • Threads extras criadas: ~{leaked * 3}')
else:
    print('ℹ️ Nenhuma conexão vazada registrada (ou dados não disponíveis)')

## 6. Resumo e Conclusões

In [ ]:
print('═' * 70)
print('                    📊 RESUMO DA ANÁLISE')
print('═' * 70)

if bad_metrics and good_metrics:
    print('\n🔴 BAD PRODUCER (Anti-pattern):')
    print(f'   • Cria NOVA conexão a cada request')
    print(f'   • NUNCA fecha as conexões')
    print(f'   • Total requests: {bad_metrics["total_requests"]}')
    print(f'   • Latência média: {bad_metrics["avg_latency_ms"]:.2f}ms')
    print(f'   • Taxa de falhas: {bad_metrics["failure_rate"]*100:.1f}%')
    print(f'   • Conexões vazadas: {bad_metrics["leaked_connections"]}')
    
    print('\n🟢 GOOD PRODUCER (Best Practice):')
    print(f'   • Reutiliza UMA ÚNICA conexão')
    print(f'   • Gerenciada pelo Spring Container')
    print(f'   • Total requests: {good_metrics["total_requests"]}')
    print(f'   • Latência média: {good_metrics["avg_latency_ms"]:.2f}ms')
    print(f'   • Taxa de falhas: {good_metrics["failure_rate"]*100:.1f}%')
    
    print('\n📈 COMPARAÇÃO:')
    if bad_metrics['avg_latency_ms'] > 0 and good_metrics['avg_latency_ms'] > 0:
        if bad_metrics['avg_latency_ms'] > good_metrics['avg_latency_ms']:
            improvement = ((bad_metrics['avg_latency_ms'] - good_metrics['avg_latency_ms']) 
                          / bad_metrics['avg_latency_ms'] * 100)
            print(f'   • Good Producer é {improvement:.1f}% mais rápido')
        else:
            # No emulador lento, bad producer pode parecer "mais rápido" por criar conexões paralelas
            print(f'   • ⚠️ No emulador lento, bad producer criou mais conexões paralelas')
    
    if bad_metrics['leaked_connections'] > 0:
        print(f'\n☠️ IMPACTO DO RESOURCE LEAK:')
        print(f'   • {bad_metrics["leaked_connections"]} conexões nunca fechadas')
        print(f'   • Em produção, isso causaria OutOfMemoryError!')

print('\n' + '═' * 70)
print('\n✅ CONCLUSÃO: Sempre use conexões gerenciadas pelo Spring Container!')
print('   Evite criar ServiceBusSenderClient manualmente em cada request.')
print('\n⚠️ NOTA: O emulador local é muito mais lento que o Azure real.')
print('   Em produção, a diferença de performance seria muito maior!')
print('═' * 70)